In [1]:
pip install neptune-client;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.4/450.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00


In [2]:
pip install foolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.2 MB/s eta 0:00:00


In [3]:
pip install neptune-notebooks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.7 MB/s eta 0:00:00
  Created wheel for neptune-notebooks: filename=neptune_notebooks-0.9.3-py2.py3-none-any.whl size=1074591 sha256=e2264e49b176dbfbdac53cda7af6d86866d942feb2eaef2163dd174f6b2c48ec
  Stored in directory: /root/.cache/pip/wheels/69/79/93/32ea88fc50a669a122b96af899a76565fdd90335aa2dfaf9b3
Successfully built neptune-notebooks


In [4]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import foolbox as fb
import torch.optim as optim


trainMode = True
constrain = True
randd = random.randint(0,1000)
#randd = 528
print(f"Current random number : {randd}")
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Cifar100Experiments/quickTestSwitch"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
from utils import *
from NormalizedConvolutions import *
from networkTraining import *
from multiClassHinge import *
from LipschitzConstant import *
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

Current random number : 955
Mounted at /content/gdrive
/content/gdrive/My Drive/Cifar100Experiments/quickTestSwitch
cuda:0


In [5]:
# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
batch_size = 128


trainset =  torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
a = 0.5
M = 1

Files already downloaded and verified
Files already downloaded and verified


In [6]:
import neptune.new as neptune

/usr/local/lib/python3.10/dist-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
<ipython-input-6-37c521bab1e7>:1: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [7]:
class Normalisation(torch.nn.Module):
    def __init__(self, means=(0.4914, 0.4822, 0.4465), stds=(0.2023, 0.1994, 0.2010)):
        super().__init__()
        assert len(means) == len(stds)
        self.means = means
        self.stds = stds

    def forward(self, x):
        return (x - torch.tensor(self.means, device=x.device).view(1, len(self.means), 1, 1)) / torch.tensor(self.stds, device=x.device).view(1, len(self.means), 1, 1)



transform_test_rob = transforms.Compose([
    transforms.ToTensor()
])
batch_size = 1024
testset_rob = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test_rob)
testloader_rob = torch.utils.data.DataLoader(testset_rob, batch_size=batch_size,
                                        shuffle=False, num_workers=2)
images, labels = next(iter(testloader_rob))
images, labels = images.to(device), labels.to(device)

Files already downloaded and verified


In [8]:
'''pip install torch-lr-finder'''

'pip install torch-lr-finder'

In [9]:
'''pip install torch-lr-finder -v --global-option="apex"'''

'pip install torch-lr-finder -v --global-option="apex"'

In [10]:
'''from torch_lr_finder import LRFinder

net = Network(3,32,64,128,4,4,4,S = 2, a = a, M=M)
net.to(device);
optimizer = optim.SGD(net.parameters(),lr=1e-7,momentum=0.9)
margin = 0.07
criterion = multiClassHingeLoss(margin = margin)
#criterion = torch.nn.MultiMarginLoss(p=1, margin=margin)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
lr_finder = LRFinder(net, optimizer, criterion, device="cuda")
lr_finder.range_test(trainloader, end_lr=1, num_iter=1000)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state'''

'from torch_lr_finder import LRFinder\n\nnet = Network(3,32,64,128,4,4,4,S = 2, a = a, M=M)\nnet.to(device);\noptimizer = optim.SGD(net.parameters(),lr=1e-7,momentum=0.9)\nmargin = 0.07\ncriterion = multiClassHingeLoss(margin = margin)\n#criterion = torch.nn.MultiMarginLoss(p=1, margin=margin)\n#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)\nlr_finder = LRFinder(net, optimizer, criterion, device="cuda")\nlr_finder.range_test(trainloader, end_lr=1, num_iter=1000)\nlr_finder.plot() # to inspect the loss-learning rate graph\nlr_finder.reset() # to reset the model and optimizer to their initial state'

In [ ]:
#marginList = [0.05, 0.07, 0.1, 0.15,0.3, 0.5]
marginList = [0.07,0.15]
#marginList = [0.15]
#marginList = [0.07]
epsilons = [0.0, 8/255, 16/255, 36/255, 0.3, 0.5, 0.6, 0.8, 1.0]
saveRobAcc = np.zeros((len(marginList),len(epsilons)))
it = 0
for num,margin in enumerate(marginList):

  print(f"\n\n Now we are working with margin = {margin}\n\n")

  net = Network(3,32,64,128,4,4,4,S = 2, a = a, M=M)
  net.to(device);

  run = neptune.init_run(
    project="geometricintegrationntnu/switchCIFAR10",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2YzM5YjI2My1kYTI2LTRhNmMtOWI5Ni1lYzlmYzBiZWZiNzIifQ==",
)  # your credentials

  #criterion = nn.CrossEntropyLoss()
  criterion = multiClassHingeLoss(margin = margin)
  #criterion = torch.nn.MultiMarginLoss(p=1, margin=margin)
  #lr = 0.01
  lr = 1e-2

  linears = list()
  dynamics = list()
  for name, m in net.named_parameters():
      #print('checking {}'.format(name))
      if 'FC.1.weight' in name:
          #print("ciao")
          linears.append(m)
      else:
          dynamics.append(m)

  optimizer = optim.SGD([{'params':dynamics},
                         {'params':linears}],lr=lr,momentum=0.9)
  #optimizer = optim.Adam(net.parameters(),lr=lr)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
  #scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=8e-3, max_lr=8e-2, step_size_up = 800, cycle_momentum=True)

  model_save_name = f'CertifiedLip_margin_{margin}.pt'
  path = F"{model_save_name}"

  if trainMode:
    loss = -1
    while loss<0:
      print(f"Training with learning rate {lr}")
      loss = train(net, margin, criterion, optimizer, scheduler, trainloader, testloader, device, epochs=100, reg=True, a=a, constrain = constrain, run = run)
      if loss<0:
        net = Network(3,32,64,128,4,4,4,S=2,a=a,M=M)
        net.to(device);
        lr/=5
        linears = list()
        dynamics = list()
        for name, m in net.named_parameters():
            print('checking {}'.format(name))
            if 'FC.1.weight' in name:
                print("ciao")
                linears.append(m)
            else:
                dynamics.append(m)

        optimizer = optim.SGD([{'params':dynamics},
                              {'params':linears}],lr=lr,momentum=0.9)
        #optimizer = optim.Adam(net.parameters(),lr=lr,weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
        #scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=8e-3, max_lr=8e-2, step_size_up = 800, cycle_momentum=True)

    run.stop()
    torch.save(net.state_dict(), path)
  else:
    net.load_state_dict(torch.load(path))

  model = nn.Sequential(Normalisation(),net).eval()
  fmodel = fb.PyTorchModel(model, bounds=(0, 1))

  acc = fb.utils.accuracy(fmodel, images, labels)

  attack = fb.attacks.L2PGD(steps=10)
  _, advs, success = attack(fmodel, images, labels, epsilons=epsilons)
  robust_accuracy = torch.mean((1-1.*success),axis=1)
  saveRobAcc[num] = robust_accuracy.detach().cpu().numpy()

  it += 1
  destination = f"updateMargin_{margin}.txt"
  np.savetxt(destination, saveRobAcc.reshape(-1,1))

  fig = plt.figure(figsize=(20,10))
  plt.plot(epsilons,robust_accuracy.detach().cpu().numpy(),'r-*',label="Experimental")
  plt.xlabel(r"$\varepsilon$",fontsize=20)
  plt.ylabel("Robust accuracy",fontsize=20)
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)
  plt.legend(fontsize=20,loc=1)
  plt.title(f"L2 robustness. Trained with margin = {margin}. Test accuracy = {round(acc * 100,2)}%",fontsize=20);

  plt.savefig(f'L2margin_{margin}.png')

fig = plt.figure(figsize=(20,10))

for i in range(len(marginList)):
  plt.plot(epsilons,saveRobAcc[i],'-*',label=f"Margin = {marginList[i]}")

plt.xlabel(r"$\varepsilon$",fontsize=20)
plt.ylabel("Robust accuracy",fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20,loc=1)
plt.title(f"L2 robustness comparison CIFAR10",fontsize=20);

plt.savefig(f'L2RobustnessComparison.png')



 Now we are working with margin = 0.07




<ipython-input-11-ec0c983120a1>:15: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://app.neptune.ai/geometricintegrationntnu/switchCIFAR10/e/SWITCH-240
Training with learning rate 0.01
Initial regularization :  tensor(59.9294, device='cuda:0', grad_fn=<AddBackward0>) 


Regularization parameter :  0.1 


epoch 0, rescalings_0 :  tensor([0.0789], device='cuda:0')
epoch 0, rescalings_1 :  tensor([0.5810], device='cuda:0')
epoch 0, rescalings_2 :  tensor([0.0413], device='cuda:0')
Loss [1, 100](epoch, minibatch):  3.9123692190647126
Loss [1, 200](epoch, minibatch):  0.7826118886470794
Loss [1, 300](epoch, minibatch):  0.6390699744224548
epoch 0, dts_0 :  tensor([0.4534, 0.2126, 0.0223, 0.0112], device='cuda:0')
epoch 0, dts_1 :  tensor([0.2408, 0.1185, 0.2924, 0.1426], device='cuda:0')
epoch 0, dts_2 :  tensor([0.0054, 0.0024, 0.3690, 0.1771], device='cuda:0')
Norms: [0.9998427629470825, 0.9986084699630737, 1.0000090599060059, 0.9995336532592773, 0.9999912977218628, 1.000805139541626, 0.9986835718154907, 1.000164270401001, 0.9995987415313721, 1.000023603439331, 0.

In [ ]:
saveRobAcc[0]

In [ ]:
fig = plt.figure(figsize=(20,10))

for i in range(len(marginList)):
  plt.plot(epsilons,saveRobAcc[i],'-*',label=f"Margin = {marginList[i]}")

plt.xlabel(r"$\varepsilon$",fontsize=20)
plt.ylabel("Robust accuracy",fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20,loc=1)
plt.title(f"L2 robustness comparison CIFAR10",fontsize=20);

plt.savefig(f'L2RobustnessComparison.png')

In [ ]:
a = torch.randn(10)
print(a)
a = torch.clamp(a,0,1)
print(a)

In [ ]:
model = nn.Sequential(Normalisation(),net).eval()
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

acc = fb.utils.accuracy(fmodel, images, labels)

attack = fb.attacks.L2PGD(steps=10)
_, advs, success = attack(fmodel, images, labels, epsilons=epsilons)
robust_accuracy = torch.mean((1-1.*success),axis=1)
saveRobAcc[num] = robust_accuracy.detach().cpu().numpy()

it += 1
destination = f"updateMargin_{margin}.txt"
np.savetxt(destination, saveRobAcc.reshape(-1,1))

fig = plt.figure(figsize=(20,10))
plt.plot(epsilons,robust_accuracy.detach().cpu().numpy(),'r-*',label="Experimental")
plt.xlabel(r"$\varepsilon$",fontsize=20)
plt.ylabel("Robust accuracy",fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20,loc=1)
plt.title(f"L2 robustness. Trained with margin = {margin}. Test accuracy = {round(acc * 100,2)}%",fontsize=20);

plt.savefig(f'L2margin_{margin}.png')